In [ ]:
%load_ext autoreload
%autoreload 2
from IPython.core.debugger import set_trace
from IPython.utils import traitlets as _traitlets

/tmp/ipykernel_1935125/3341169811.py:4: UserWarning: IPython.utils.traitlets has moved to a top-level traitlets package.
  from IPython.utils import traitlets as _traitlets


<h1><center> Fine tune learner </center></h1>

In this notebook, we show how we can fine tune an already trained learner. The idea is to show a learner more training examples with a low learning rate to improve it without loosing the trained weights already obtained.

In [ ]:
import glob
import os
import pickle
import tempfile
import warnings
from pathlib import Path
from random import sample
from typing import List, Tuple

import numpy as np
import pandas as pd
from fastcore.basics import *
from fastcore.foundation import *
from fastcore.xtras import load_pickle, save_pickle
from progressbar import progressbar
from tsai.all import *

from footSeq.datastruct.core import *
from footSeq.model.learner import *
from footSeq.plots import *

device = default_device()
computer_setup()

os             : Linux-5.11.0-46-generic-x86_64-with-glibc2.31
python         : 3.9.6
tsai           : 0.2.25
fastai         : 2.5.3
fastcore       : 1.3.26
torch          : 1.9.1
n_cpus         : 16
device         : cuda (NVIDIA GeForce GTX 1050 Ti)


# Load a Trained learner

In [ ]:
model_name = "LSTM_FCN_bidir-True_layers-2_no_goal_prop-2_full_data"
base_path = Path(".")

learn = load_all(
    path=base_path / "models" / model_name,
    dls_fname="dls",
    model_fname="model",
    learner_fname="learner",
    verbose=True,
)

Learner loaded:
path          = 'models/LSTM_FCN_bidir-True_layers-2_no_goal_prop-2_full_data'
model_fname   = 'model.pth'
learner_fname = 'learner.pkl'


# Predict one game

In [ ]:
from footSeq.config.mongo import mongo_init

mongo_init("prod_atlas")

game_id = 2162758

game_probs = learn.predict_game(game_id, save=True)

game_probs.to_csv("/home/tarak/Downloads/action_values_" + str(game_id) +".csv")

# Investigate Learner attributes

This option is inspired from [This blog](https://www.geeksforgeeks.org/how-to-get-a-list-of-class-attributes-in-python/):

In [ ]:
import inspect

# members of an object 
for i in inspect.getmembers(learn):
      
    # to remove private and protected
    # functions
    if not i[0].startswith('_'):
          
        # To remove other methods that
        # doesnot start with a underscore
        if not inspect.ismethod(i[1]): 
            print(i)

('T_destination', ~T_destination)
('arch', <class 'tsai.models.RNN_FCN.LSTM_FCN'>)
('body', LSTM_FCN(
  (rnn): LSTM(74, 100, num_layers=2, batch_first=True, bidirectional=True)
  (rnn_dropout): Dropout(p=0.8, inplace=False)
  (shuffle): Permute(dims=0, 2, 1)
  (convblock1): ConvBlock(
    (0): Conv1d(74, 128, kernel_size=(7,), stride=(1,), padding=(3,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock2): ConvBlock(
    (0): Conv1d(128, 256, kernel_size=(5,), stride=(1,), padding=(2,), bias=False)
    (1): BatchNorm1d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (convblock3): ConvBlock(
    (0): Conv1d(256, 128, kernel_size=(3,), stride=(1,), padding=(1,), bias=False)
    (1): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU()
  )
  (gap): GAP1d(
    (gap): AdaptiveAvgPool1d(output_size=1)
    (flatten): Flatten

Now, we load all training files:

# Prepare data-loaders

We will split our files into train/valid data-loaders. `Dataloaders` will be saved on the disk and will be re-used later eventually: 

In [ ]:
metadata_path = Path("./data")
files_list_path = metadata_path / "file_list.pkl"
model_path = Path("./models/LSTM_FCN_bidir-True_layers-2_no_goal_prop-2_fine_tuned")

all_files = L(load_pickle(files_list_path))

In [ ]:
import logging

def chunker(seq, size):
    return (seq[pos:pos + size] for pos in range(0, len(seq), size))

chunk_size = int(len(all_files) / 20)
chunks = chunker(all_files, chunk_size)
n_chunks = len(list(chunks))
chunk = 0

for chunk_files in chunker(all_files, chunk_size):
    splits_files = RandomSplitter(seed=42)(chunk_files)
    chunk +=1
    if chunk == 13:
        logging.info(f"working on chunk:{chunk}/{n_chunks} !")
        new_train_dl = learn.dls.test_dl(chunk_files[splits_files[0]], with_labels=True, shuffle=True, drop_last=True)
        new_valid_dl = learn.dls.test_dl(chunk_files[splits_files[1]], with_labels=True)
        torch.save(new_train_dl, model_path / f"train_dls-{chunk}-{n_chunks}.pth")
        torch.save(new_valid_dl, model_path / f"valid_dls-{chunk}-{n_chunks}.pth")

2022-01-23 10:18:30 - working on chunk:13/21 !
